In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/LDA_PROJECT_FOLDER/processed_text (1).zip"
extract_to = "/content/lda_model"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# See what's inside
print(os.listdir(extract_to))


['processed_text.csv']


In [ ]:
import pandas as pd

csv_path = "/content/lda_model/processed_text.csv"
df = pd.read_csv(csv_path)

# Preview the data
df.head()


,Unnamed: 0,Text,Cleaned_Text,lemmatized_text
0,0,I have bought several of the Vitality canned d...,bought several vitality canned dog food produc...,buy several vitality can dog food product find...
1,1,Product arrived labeled as Jumbo Salted Peanut...,product arrived labeled jumbo salted peanutsth...,product arrive label jumbo salt peanutsthe pea...
2,2,This is a confection that has been around a fe...,confection around centuries light pillowy citr...,confection around century light pillowy citrus...
3,3,If you are looking for the secret ingredient i...,looking secret ingredient robitussin believe f...,look secret ingredient robitussin believe find...
4,4,Great taffy at a great price. There was a wid...,great taffy great price wide assortment yummy ...,great taffy great price wide assortment yummy ...


In [ ]:
!pip install numpy==1.24.4 scipy==1.10.1 --upgrade --force-reinstall



  Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached scipy-1.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
Using cached scipy-1.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blosc2 3.3.2 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
scikit-image 0.25.2 requires scipy>=1.11

In [ ]:
import os
os.listdir('/content/drive/MyDrive/LDA_PROJECT_FOLDER')


['BTech_project_tokenize.ipynb',
 'BTech_project_LDA.ipynb',
 'BTech_LDA_model.gensim',
 'processed_text (1).zip',
 'BTech_LDA_model.gensim.expElogbeta.npy',
 'BTech_LDA_model.gensim.id2word',
 'BTech_LDA_model.gensim.state']

In [ ]:
# Install and import everything
!pip install -q gensim nltk

import pandas as pd
import numpy as np
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from ast import literal_eval

# Download VADER
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# Load the CSV (assumes file was extracted from ZIP to this path)
df = pd.read_csv("/content/lda_model/processed_text.csv")

# Tokenize the lemmatized text column
df['tokenized'] = df['lemmatized_text'].apply(lambda x: x.split() if isinstance(x, str) else [])


# Prepare corpus and dictionary
dictionary = Dictionary.load('/content/drive/MyDrive/LDA_PROJECT_FOLDER/BTech_LDA_model.gensim.id2word')
corpus = [dictionary.doc2bow(text) for text in df['tokenized']]

# Load the pretrained LDA model
lda_model = LdaModel.load("/content/drive/MyDrive/LDA_PROJECT_FOLDER/BTech_LDA_model.gensim")


# Assign dominant topic
def get_dominant_topic(bow):
    topics = lda_model.get_document_topics(bow)
    if topics:
        return max(topics, key=lambda x: x[1])[0]
    return -1

df['bow'] = corpus
df['Dominant_Topic'] = df['bow'].apply(get_dominant_topic)

# Perform VADER sentiment analysis
def get_sentiment(text):
    score = sia.polarity_scores(text)
    compound = score['compound']
    if compound >= 0.05:
        return 'Positive'
    elif compound <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

df['Sentiment'] = df['text'].apply(get_sentiment)

# Compute sentiment percentage per topic
topic_sentiment_pct = pd.crosstab(
    df['Dominant_Topic'],
    df['Sentiment'],
    normalize='index'
) * 100

# Rename columns and reset index
topic_sentiment_pct = topic_sentiment_pct.rename(columns={
    'Positive': 'Positive_%',
    'Neutral': 'Neutral_%',
    'Negative': 'Negative_%'
}).reset_index()

# Merge sentiment percentages with main DataFrame
df = df.merge(topic_sentiment_pct, on='Dominant_Topic', how='left')

# Final output table
final_df = df[['text', 'Dominant_Topic', 'Positive_%', 'Neutral_%', 'Negative_%']]
final_df.head()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


KeyError: 'text'

In [ ]:
from gensim.models import LdaModel

In [ ]:
lda_model = LdaModel.load("/content/drive/MyDrive/LDA_PROJECT_FOLDER/BTech_LDA_model.gensim")

In [ ]:
# Print all topics (default: top 10 words per topic)
topics = lda_model.print_topics(num_topics=-1)  # -1 means all topics
for topic_id, topic in topics:
    print(f"Topic {topic_id}: {topic}")

Topic 0: 0.052*"small" + 0.050*"piece" + 0.049*"chew" + 0.041*"size" + 0.027*"tooth" + 0.027*"large" + 0.023*"stick" + 0.022*"bold" + 0.022*"hard" + 0.019*"get"
Topic 1: 0.048*"bag" + 0.014*"open" + 0.014*"package" + 0.013*"plastic" + 0.012*"container" + 0.011*"oz" + 0.011*"small" + 0.011*"come" + 0.010*"one" + 0.009*"ounce"
Topic 2: 0.058*"drink" + 0.042*"flavor" + 0.040*"taste" + 0.037*"water" + 0.034*"sugar" + 0.026*"like" + 0.023*"juice" + 0.022*"coconut" + 0.022*"sweet" + 0.019*"cherry"
Topic 3: 0.077*"oil" + 0.070*"sauce" + 0.046*"hot" + 0.036*"use" + 0.028*"olive" + 0.027*"seed" + 0.025*"pepper" + 0.019*"bottle" + 0.017*"heat" + 0.013*"jar"
Topic 4: 0.090*"cup" + 0.054*"bean" + 0.053*"milk" + 0.047*"make" + 0.041*"use" + 0.039*"cream" + 0.033*"vanilla" + 0.028*"ice" + 0.025*"pod" + 0.024*"starbuck"
Topic 5: 0.031*"chicken" + 0.030*"add" + 0.029*"meal" + 0.023*"beef" + 0.022*"noodle" + 0.019*"cook" + 0.019*"soup" + 0.019*"meat" + 0.018*"food" + 0.018*"broccoli"
Topic 6: 0.044*"lo

In [ ]:
df

,Unnamed: 0,Text,Cleaned_Text,lemmatized_text
0,0,I have bought several of the Vitality canned d...,bought several vitality canned dog food produc...,"['buy', 'several', 'vitality', 'can', 'dog', '..."
1,1,Product arrived labeled as Jumbo Salted Peanut...,product arrived labeled jumbo salted peanutsth...,"['product', 'arrive', 'label', 'jumbo', 'salt'..."
2,2,This is a confection that has been around a fe...,confection around centuries light pillowy citr...,"['confection', 'around', 'century', 'light', '..."
3,3,If you are looking for the secret ingredient i...,looking secret ingredient robitussin believe f...,"['look', 'secret', 'ingredient', 'robitussin',..."
4,4,Great taffy at a great price. There was a wid...,great taffy great price wide assortment yummy ...,"['great', 'taffy', 'great', 'price', 'wide', '..."
...,...,...,...,...
99995,99995,I just love it and will buy another box when I...,love buy another box im done last one,"['love', 'buy', 'another', 'box', 'I', 'm', 'd..."
99996,99996,My late father in law used to have a rating sy...,late father law used rating system meals parti...,"['late', 'father', 'law', 'use', 'rating', 'sy..."
99997,99997,This is my favorite brand of Korean ramen. It ...,favorite brand korean ramen spicy used eating ...,"['favorite', 'brand', 'korean', 'raman', 'spic..."
99998,99998,"I do like these noodles although, to say they ...",like noodles although say spicy somewhat under...,"['like', 'noodle', 'although', 'say', 'spicy',..."


In [ ]:
df=df[['lemmatized_text']]
df

,lemmatized_text
0,"['buy', 'several', 'vitality', 'can', 'dog', '..."
1,"['product', 'arrive', 'label', 'jumbo', 'salt'..."
2,"['confection', 'around', 'century', 'light', '..."
3,"['look', 'secret', 'ingredient', 'robitussin',..."
4,"['great', 'taffy', 'great', 'price', 'wide', '..."
...,...
99995,"['love', 'buy', 'another', 'box', 'I', 'm', 'd..."
99996,"['late', 'father', 'law', 'use', 'rating', 'sy..."
99997,"['favorite', 'brand', 'korean', 'raman', 'spic..."
99998,"['like', 'noodle', 'although', 'say', 'spicy',..."


In [ ]:
import ast
df['lemmatized_text'] = df['lemmatized_text'].apply(ast.literal_eval)
df

<ipython-input-21-cf5c8d3f5edf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lemmatized_text'] = df['lemmatized_text'].apply(ast.literal_eval)


,lemmatized_text
0,"[buy, several, vitality, can, dog, food, produ..."
1,"[product, arrive, label, jumbo, salt, peanutst..."
2,"[confection, around, century, light, pillowy, ..."
3,"[look, secret, ingredient, robitussin, believe..."
4,"[great, taffy, great, price, wide, assortment,..."
...,...
99995,"[love, buy, another, box, I, m, do, last, one]"
99996,"[late, father, law, use, rating, system, meal,..."
99997,"[favorite, brand, korean, raman, spicy, use, e..."
99998,"[like, noodle, although, say, spicy, somewhat,..."


In [ ]:
# Load the CSV (assumes file was extracted from ZIP to this path)
corpus = [dictionary.doc2bow(text) for text in df['lemmatized_text']]

In [ ]:
# Assign dominant topic
def get_dominant_topic(bow):
    topics = lda_model.get_document_topics(bow)
    if topics:
        return max(topics, key=lambda x: x[1])[0]
    return -1

df['bow'] = corpus
df['Dominant_Topic'] = df['bow'].apply(get_dominant_topic)

<ipython-input-23-34d811822657>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bow'] = corpus
<ipython-input-23-34d811822657>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Dominant_Topic'] = df['bow'].apply(get_dominant_topic)


In [ ]:
df

,lemmatized_text,bow,Dominant_Topic
0,"[buy, several, vitality, can, dog, food, produ...","[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...",8
1,"[product, arrive, label, jumbo, salt, peanutst...","[(13, 2), (20, 1), (21, 1), (22, 1), (23, 1), ...",12
2,"[confection, around, century, light, pillowy, ...","[(35, 1), (36, 1), (37, 1), (38, 1), (39, 1), ...",21
3,"[look, secret, ingredient, robitussin, believe...","[(4, 1), (7, 1), (10, 1), (68, 1), (69, 1), (7...",2
4,"[great, taffy, great, price, wide, assortment,...","[(67, 1), (83, 1), (84, 1), (85, 1), (86, 2), ...",11
...,...,...,...
99995,"[love, buy, another, box, I, m, do, last, one]","[(1, 1), (105, 1), (123, 1), (137, 1), (141, 1...",10
99996,"[late, father, law, use, rating, system, meal,...","[(4, 1), (9, 1), (11, 2), (46, 1), (56, 1), (7...",19
99997,"[favorite, brand, korean, raman, spicy, use, e...","[(6, 1), (32, 1), (76, 2), (86, 1), (94, 1), (...",5
99998,"[like, noodle, although, say, spicy, somewhat,...","[(9, 1), (10, 1), (19, 1), (108, 1), (184, 1),...",19


In [ ]:
df1=pd.read_csv("/content/lda_model/processed_text.csv")
df1=df1[['lemmatized_text']]
df1

,lemmatized_text
0,buy several vitality can dog food product find...
1,product arrive label jumbo salt peanutsthe pea...
2,confection around century light pillowy citrus...
3,look secret ingredient robitussin believe find...
4,great taffy great price wide assortment yummy ...
...,...
99995,love buy another box I m do last one
99996,late father law use rating system meal particu...
99997,favorite brand korean raman spicy use eat spic...
99998,like noodle although say spicy somewhat unders...


In [ ]:
df_combined = pd.concat([df1[['lemmatized_text']].reset_index(drop=True), df], axis=1)
df_combined

,lemmatized_text,lemmatized_text,bow,Dominant_Topic
0,buy several vitality can dog food product find...,"[buy, several, vitality, can, dog, food, produ...","[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...",8
1,product arrive label jumbo salt peanutsthe pea...,"[product, arrive, label, jumbo, salt, peanutst...","[(13, 2), (20, 1), (21, 1), (22, 1), (23, 1), ...",12
2,confection around century light pillowy citrus...,"[confection, around, century, light, pillowy, ...","[(35, 1), (36, 1), (37, 1), (38, 1), (39, 1), ...",21
3,look secret ingredient robitussin believe find...,"[look, secret, ingredient, robitussin, believe...","[(4, 1), (7, 1), (10, 1), (68, 1), (69, 1), (7...",2
4,great taffy great price wide assortment yummy ...,"[great, taffy, great, price, wide, assortment,...","[(67, 1), (83, 1), (84, 1), (85, 1), (86, 2), ...",11
...,...,...,...,...
99995,love buy another box I m do last one,"[love, buy, another, box, I, m, do, last, one]","[(1, 1), (105, 1), (123, 1), (137, 1), (141, 1...",10
99996,late father law use rating system meal particu...,"[late, father, law, use, rating, system, meal,...","[(4, 1), (9, 1), (11, 2), (46, 1), (56, 1), (7...",19
99997,favorite brand korean raman spicy use eat spic...,"[favorite, brand, korean, raman, spicy, use, e...","[(6, 1), (32, 1), (76, 2), (86, 1), (94, 1), (...",5
99998,like noodle although say spicy somewhat unders...,"[like, noodle, although, say, spicy, somewhat,...","[(9, 1), (10, 1), (19, 1), (108, 1), (184, 1),...",19


In [ ]:
df_combined.to_csv("lda_topic_distributions_with_scores.csv", index=False)

from google.colab import files
files.download('lda_topic_distributions_with_scores.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
best_model.save("/content/drive/MyDrive/LDA_PROJECT_FOLDER/BTech_LDA_model.gensim")


NameError: name 'best_model' is not defined

In [ ]:
!pip install gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 8.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 requ

In [ ]:

from gensim.models import LdaModel

# Load the previously saved best model from Drive
best_model = LdaModel.load("/content/drive/MyDrive/LDA_PROJECT_FOLDER/BTech_LDA_model.gensim")


ModuleNotFoundError: No module named 'numpy.rec'

In [ ]:
# Step 1: Automatically generate topic names using top 3 keywords per topic
def generate_topic_labels(lda_model, topn=3):
    topic_names = {}
    for topic_id in range(lda_model.num_topics):
        top_words = [word for word, _ in lda_model.show_topic(topic_id, topn=topn)]
        label = " ".join(top_words)
        topic_names[topic_id] = label
    return topic_names

# Step 2: Generate the labels from best_model
topic_names = generate_topic_labels(best_model, topn=3)

# Step 3: Map topic numbers to names
df['Topic_Name'] = df['Dominant_Topic'].map(topic_names)


NameError: name 'best_model' is not defined